In [ ]:
#%pip install panel

In [1]:
import pickle
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np

In [2]:
with open('book_topics.pkl', 'rb') as picklefile:
    book_topics= pickle.load(picklefile)

with open('authors_df.pkl', 'rb') as picklefile:
    authors_df= pickle.load(picklefile)       
    
with open('book_ratings_sim.pkl', 'rb') as picklefile:
    book_ratings_sim= pickle.load(picklefile)

In [3]:
import pandas as pd
books_data = pd.read_csv('books.csv')

In [4]:
df = book_topics.copy()
df['title'] = books_data.title
df['authors'] = books_data.authors.apply(lambda x: x.split(',')[0])
df.head()

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,title,authors
0,0.000048,0.010749,0.999998,0.000348,0.999999,0.000001,0.001318,0.867652,0.000106,0.660895,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins
1,0.022753,0.000001,0.999983,0.000120,0.999999,0.997581,0.999415,0.999999,0.999999,0.066583,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling
2,0.000008,0.000004,0.000026,0.000020,0.142661,0.000024,0.000053,0.000402,0.000570,0.005461,"Twilight (Twilight, #1)",Stephenie Meyer
3,0.999999,0.999999,0.000014,0.000069,0.001820,0.999651,0.000001,0.005274,0.000107,0.273786,To Kill a Mockingbird,Harper Lee
4,0.000024,0.999889,0.000250,0.000187,0.000167,0.999999,0.000131,0.000810,0.000106,0.115376,The Great Gatsby,F. Scott Fitzgerald


In [5]:
poc_authors = list(authors_df[authors_df.poc == 1].index.unique())

In [6]:
df['poc'] =df['authors'].apply(lambda x: 1 if x in poc_authors else 0)
df.head()

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,title,authors,poc
0,0.000048,0.010749,0.999998,0.000348,0.999999,0.000001,0.001318,0.867652,0.000106,0.660895,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,0
1,0.022753,0.000001,0.999983,0.000120,0.999999,0.997581,0.999415,0.999999,0.999999,0.066583,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling,0
2,0.000008,0.000004,0.000026,0.000020,0.142661,0.000024,0.000053,0.000402,0.000570,0.005461,"Twilight (Twilight, #1)",Stephenie Meyer,0
3,0.999999,0.999999,0.000014,0.000069,0.001820,0.999651,0.000001,0.005274,0.000107,0.273786,To Kill a Mockingbird,Harper Lee,0
4,0.000024,0.999889,0.000250,0.000187,0.000167,0.999999,0.000131,0.000810,0.000106,0.115376,The Great Gatsby,F. Scott Fitzgerald,0


In [7]:
#Here we create similarity matrix using topics from topic modeling
from sklearn.metrics.pairwise import cosine_similarity
sim_matrix = cosine_similarity(df[[f'topic_{i}' for i in range(1,11)]])
print(sim_matrix.shape)
book_summary_sim = pd.DataFrame(sim_matrix)
book_summary_sim.head()

(9955, 9955)


,0,1,2,3,4,5,6,7,8,9,...,9945,9946,9947,9948,9949,9950,9951,9952,9953,9954
0,1.000000,0.665944,0.575127,0.063277,0.034775,0.525462,0.012177,0.425657,0.526519,0.188129,...,0.039015,0.927543,0.005013,0.568787,0.411102,0.569277,0.010963,0.041627,0.698042,0.477450
1,0.665944,1.000000,0.412095,0.243438,0.289697,0.609275,0.298238,0.382651,0.408723,0.222879,...,0.263688,0.708500,0.289180,0.453885,0.070944,0.602416,0.029879,0.039897,0.493749,0.607644
2,0.575127,0.412095,1.000000,0.007162,0.003368,0.071830,0.001606,0.017514,0.501177,0.021169,...,0.061281,0.579476,0.002960,0.997610,0.095866,0.524585,0.003139,0.071727,0.021290,0.376640
3,0.063277,0.243438,0.007162,1.000000,0.816468,0.305521,0.425449,0.584030,0.287519,0.648931,...,0.986473,0.002617,0.403587,0.029672,0.161786,0.012765,0.685346,0.000729,0.385301,0.666634
4,0.034775,0.289697,0.003368,0.816468,1.000000,0.376352,0.507076,0.354566,0.352907,0.393796,...,0.811256,0.000717,0.498466,0.033554,0.088326,0.006569,0.440159,0.000469,0.067707,0.536577


In [8]:
poc_indeces = list(df[df.poc == 1].index)
not_poc_indeces = list(df[df.poc != 1].index)

In [9]:
#Here we make a new summary-based similarity matrix where columns correspond only to POC authors
book_summary_sim_new = pd.DataFrame([])
for i in poc_indeces:
    book_summary_sim_new[i] = book_summary_sim[book_summary_sim.columns[i]]
book_summary_sim_new.head()

,10,66,93,117,126,192,197,210,232,267,...,9690,9724,9728,9767,9768,9808,9822,9843,9846,9861
0,0.134658,0.215567,0.018948,0.469868,0.258152,0.396484,0.406408,0.250246,0.263493,0.587014,...,0.527171,0.560395,0.021030,0.604682,0.536334,0.402012,0.635278,0.581127,0.000402,0.657496
1,0.366445,0.259729,0.003559,0.466257,0.368842,0.303518,0.408400,0.541781,0.307224,0.536590,...,0.383930,0.410546,0.086047,0.707800,0.657959,0.579969,0.558619,0.677084,0.408382,0.759503
2,0.013698,0.022020,0.001565,0.015640,0.003827,0.000220,0.021928,0.002091,0.027289,0.017378,...,0.938060,0.999246,0.001485,0.580936,0.016080,0.003584,0.462380,0.425197,0.004046,0.062710
3,0.745928,0.750455,0.578357,0.835957,0.759351,0.414235,0.676351,0.750569,0.512630,0.754549,...,0.003117,0.001133,0.674793,0.002722,0.705004,0.006753,0.582106,0.530357,0.000183,0.326344
4,0.882485,0.863588,0.710829,0.670295,0.619827,0.013683,0.776501,0.618362,0.554462,0.619057,...,0.002443,0.000223,0.830991,0.000624,0.562397,0.005271,0.667425,0.608137,0.000140,0.352243


In [10]:
#Now we do the same for the ratings-based similarity matrix
book_ratings_sim_new = pd.DataFrame([])
for i in poc_indeces:
    book_ratings_sim_new[i] = book_ratings_sim[book_ratings_sim.columns[i]]
book_ratings_sim_new.head()

,10,66,93,117,126,192,197,210,232,267,...,9690,9724,9728,9767,9768,9808,9822,9843,9846,9861
0,0.380074,0.298266,0.154101,0.231795,0.154607,0.157799,0.207000,0.148572,0.147344,0.184701,...,0.028695,0.037161,0.018077,0.023283,0.027799,0.026250,0.015878,0.038049,0.022227,0.018961
1,0.377382,0.262126,0.202824,0.244721,0.172149,0.146143,0.208829,0.151797,0.166965,0.164360,...,0.030214,0.028966,0.013919,0.022401,0.020701,0.024050,0.011271,0.030813,0.014795,0.023508
2,0.312586,0.239782,0.114091,0.203018,0.101318,0.087881,0.167435,0.094027,0.113842,0.111625,...,0.019704,0.020863,0.018027,0.006393,0.023131,0.021769,0.005863,0.020907,0.012719,0.009084
3,0.455294,0.330596,0.270491,0.317079,0.201656,0.176067,0.321271,0.179782,0.239470,0.199011,...,0.029841,0.020846,0.018651,0.018264,0.011858,0.022726,0.028585,0.014147,0.021427,0.031017
4,0.401154,0.279369,0.296553,0.290490,0.202484,0.178955,0.279899,0.176883,0.257704,0.208752,...,0.028745,0.019415,0.014102,0.022038,0.014276,0.021127,0.020070,0.009537,0.021927,0.041629


In [240]:
def title_to_id():
    book_title = input('Search for a title ')
    book_title = book_title.strip().lower()
    flag = False
    while flag == False:
        options = []
        for book in df.title.unique():
            if book_title in book.lower() and len(book_title) > 3:
                options.append(book)
        for i in options:
            print(i, '\n')
        input_ = input('Is it any of these? Enter title if yes. Otherwise, enter no.')
        if input_ != 'no' and input_ != 'No':
            return books_data.set_index('title').loc[input_].book_id
            flag = True
        else:
            input_ = input('Do you want to type again? Or exit? Type Y or N.')
            if input_ == 'N':
                flag = True
            else: 
                input_ = input('Type another title.')
                book_title = input_

In [241]:
title_to_id()

Search for a title 


KeyboardInterrupt: 

In [19]:
#Function that will interact with widget
def f(a):
        print(f'Content: {a}%, Ratings: {100-a}%'.format(a))

In [330]:
def k_similar_poc(book_id, k):
    book_index = book_id - 1
    '''Input the index of a book to get the k-most similar books written by a POC.'''
    if book_index in poc_indeces:
        print(f'This book is written by {df.iloc[book_index][11]}, who is a POC!')
            
    print(f'Books similar to {df.iloc[book_index][10]}:')
    book_recs = []
    similarities_content = book_summary_sim_new.iloc[book_index]
    similarities_ratings = book_ratings_sim_new.iloc[book_index]
    flag = True
    while flag == True:
        content_percent = input('''Choose what percentage of content you want recs to be based on.   
        Pick a number from 0 to 100. --->''')
        ready = input(f' Is {content_percent} correct? Input Y or N')
        if ready == 'Y':
            flag = False
    content_percent = int(content_percent)
            
    print(content_percent)
    ratings_percent = 100 - content_percent
    
    similarities = content_percent * similarities_content + ratings_percent * similarities_ratings
    top_indices = similarities.unique().argsort()[-k:][::-1]
    recommended_book_indices =  similarities.index[top_indices]
    
    for index in recommended_book_indices:
        book_recs.append(f'{df.iloc[index][10]}, by {df.iloc[index][11]}')
    return book_recs

In [331]:
k_similar_poc(3, 5)

Books similar to Twilight (Twilight, #1):
Choose what percentage of content you want recs to be based on.   
        Pick a number from 0 to 100. --->56
 Is 56 correct? Input Y or NY
56


['Black Boy, by Richard Wright',
 'Addicted, by Zane',
 'The Boy Who Harnessed the Wind: Creating Currents of Electricity and Hope, by William Kamkwamba',
 'Half Girlfriend, by Chetan Bhagat',
 'UnDivided (Unwind, #4), by Neal Shusterman']

In [334]:
#still working on this
def my_widget():
    global result
    print('Choose what percentage of content you want to affect your recs.')
    def w(percent):
        #global output_slider_variable
        print(f'{percent}% content, {100-percent}% ratings')
        output_slider_variable.value = str(percent)
    output_slider_variable = widgets.Text()    
    interact(w, percent = widgets.IntSlider(min=0, max=100, step=1, value=50));
    input_ = input('Enter when done')
    result = int(output_slider_variable.value)
    return result   

In [336]:
my_widget()

Choose what percentage of content you want to affect your recs.


interactive(children=(IntSlider(value=50, description='percent'), Output()), _dom_classes=('widget-interact',)…

KeyboardInterrupt: 